<a href="https://colab.research.google.com/github/deepanshu97parnami/Colab/blob/main/MLAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.svm import SVC
import sklearn
from sklearn import svm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

pd.options.mode.chained_assignment = None
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/file/d/1Ou9NRiNOxQ1h5CgQ4VPk-_q33bkdOuCR/view?usp=sharing'
  
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_v2.csv')  

train = pd.read_csv('train_v2.csv')



link = 'https://drive.google.com/file/d/1s9ujwXgLwWkR7wtvhSMl5Ngv7ug-BxVM/view?usp=sharing'
  
  
# to get the id part of the file
id1 = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('sample_submission_v2.csv')  

sample_submission = pd.read_csv('sample_submission_v2.csv')



link = 'https://drive.google.com/file/d/1PQ89UVrm9-02TJRuULM62zTTmLrkbN3Q/view?usp=sharing'
  
  
# to get the id part of the file
id2 = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('transactions_v2.csv')  

transactions = pd.read_csv('transactions_v2.csv')





link = 'https://drive.google.com/file/d/1y8nDre1OyvmPW4P9a3q61-Ui9ktzV_Ix/view?usp=sharing'
  
  
# to get the id part of the file
id3 = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id3}) 
downloaded.GetContentFile('user_logs_v2.csv')  

user_logs = pd.read_csv('user_logs_v2.csv')





link = 'https://drive.google.com/file/d/1K4BiItsbuFwmhdW82riPlkLYSDlFeADD/view?usp=sharing'
  

id4 = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id4}) 
downloaded.GetContentFile('members_v3.csv')  

members = pd.read_csv('members_v3.csv')




# set the options so the output format can be displayed correctly
pd.set_option('expand_frame_repr', True)
pd.set_option('display.max_rows', 30000000)
pd.set_option('display.max_columns', 100)

# check the number of duplicate accounts in each table
train.duplicated('msno').sum()
sample_submission.duplicated('msno').sum()
transactions.duplicated('msno').sum()
user_logs.duplicated('msno').sum()
members.duplicated('msno').sum()

# returns the max value of numerical variables and membership_expire_date
# returns the min value of transaction date
# returns the mode of ordinal variable and dummy variables, if multiple values share the same frequency, keep the first one
transactions_v2 = transactions.groupby('msno', as_index = False).agg({'payment_method_id': lambda x:x.value_counts().index[0], 'payment_plan_days': 'max', 'plan_list_price': 'max',
                                       'actual_amount_paid': 'max', 'is_auto_renew': lambda x:x.value_counts().index[0], 'transaction_date': 'min', 'membership_expire_date': 'max',
                                       'is_cancel': lambda x:x.value_counts().index[0]})

# returns the max value of date and number of unique songs
# returns the sum of other variables
user_logs_v2 = user_logs.groupby('msno', as_index = False).agg({'date': 'max', 'num_25': 'sum', 'num_50': 'sum', 'num_75': 'sum',
                                 'num_985': 'sum', 'num_100': 'sum', 'num_unq': 'max', 'total_secs': 'sum'})

# calculate the percentage of number of songs played within certain period
user_logs_v2['percent_25'] = user_logs_v2['num_25']/(user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100'])
user_logs_v2['percent_50'] = user_logs_v2['num_50']/(user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100'])
user_logs_v2['percent_100'] = (user_logs_v2['num_985']+user_logs_v2['num_100'])/(user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100'])

# drop useless variables
user_logs_v3 = user_logs_v2.drop(columns = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100'])

# merge between different tables for modelling purpose
dataset_train = train.merge(members, on = 'msno', how = 'left').merge(transactions_v2, on = 'msno', how = 'left').merge(user_logs_v3, on = 'msno', how = 'left')
dataset_train.dtypes

# date in csv will be recognized as float in python
# this value needs to be converted back to date
dataset_train['registration_init_time'] = pd.to_datetime(dataset_train['registration_init_time'], format = '%Y%m%d')
dataset_train['transaction_date'] = pd.to_datetime(dataset_train['transaction_date'], format = '%Y%m%d')
dataset_train['membership_expire_date'] = pd.to_datetime(dataset_train['membership_expire_date'], format = '%Y%m%d')
dataset_train['date'] = pd.to_datetime(dataset_train['date'], format = '%Y%m%d')

# check the maximum of datetime value
dataset_train.select_dtypes(include = ['datetime64[ns]']).max()

# create new day columns for modelling purpose
dataset_train['registration_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['registration_init_time']).astype('timedelta64[D]')
dataset_train['transaction_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['transaction_date']).astype('timedelta64[D]')
dataset_train['membership_expire_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['membership_expire_date']).astype('timedelta64[D]')
dataset_train['last_play_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['date']).astype('timedelta64[D]')

# check the distribution of age due to the documentation explanation
dataset_train['bd'].value_counts()

# remove gender and age since missing value or incorrect value is over 50%
dataset_train_v2 = dataset_train.drop(columns = ['msno', 'gender', 'bd', 'registration_init_time', 'transaction_date', 'membership_expire_date', 'date'])
dataset_train_v2.dtypes

# check the number of missing values in each column
dataset_train_v2.isna().sum()

# Handle missing value of part of numeric columns by using mode
def replacemode(i):
    dataset_train_v2[i] = dataset_train_v2[i].fillna(dataset_train_v2[i].value_counts().index[0])
    return 

replacemode('city')
replacemode('registered_via')
replacemode('payment_method_id')
replacemode('payment_plan_days')
replacemode('is_auto_renew')
replacemode('is_cancel')

# Handle missing value of part of numeric columns by using mean
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
def replacemean(i):
    dataset_train_v2[i] = mean_imputer.fit_transform(dataset_train_v2[[i]])
    return 

replacemean('plan_list_price')
replacemean('actual_amount_paid')
replacemean('num_unq')
replacemean('total_secs')
replacemean('percent_25')
replacemean('percent_50')
replacemean('percent_100')
replacemean('registration_day')
replacemean('transaction_day')
replacemean('membership_expire_day')
replacemean('last_play_day')

# Handle outliers by using capping
def replaceoutlier(i):
    mean, std = np.mean(dataset_train_v2[i]), np.std(dataset_train_v2[i])
    cut_off = std*3
    lower, upper = mean - cut_off, mean + cut_off
    dataset_train_v2[i][dataset_train_v2[i] < lower] = lower
    dataset_train_v2[i][dataset_train_v2[i] > upper] = upper
    return

replaceoutlier('plan_list_price')
replaceoutlier('actual_amount_paid')
replaceoutlier('num_unq')
replaceoutlier('total_secs')
replaceoutlier('percent_25')
replaceoutlier('percent_50')
replaceoutlier('percent_100')
replaceoutlier('registration_day')
replaceoutlier('transaction_day')
replaceoutlier('membership_expire_day')
replaceoutlier('last_play_day')

dataset_train_v2.dtypes
dataset_train_v2.describe()

# convert categorical variables into string
dataset_train_v2.iloc[:, 1:4] = dataset_train_v2.iloc[:, 1:4].astype(str)

# replace discrete features with historical churn rate
city_mean = pd.DataFrame(dataset_train_v2.groupby('city')['is_churn'].mean().reset_index())
city_mean.rename(columns = {'is_churn': 'city_mean'}, inplace=True)
register_mean = pd.DataFrame(dataset_train_v2.groupby('registered_via')['is_churn'].mean().reset_index())
register_mean.rename(columns = {'is_churn': 'register_mean'}, inplace=True)
payment_mean = pd.DataFrame(dataset_train_v2.groupby('payment_method_id')['is_churn'].mean().reset_index())
payment_mean.rename(columns = {'is_churn': 'payment_mean'}, inplace=True)

dataset_train_v3 = dataset_train_v2.merge(city_mean, on = 'city', how = 'left').merge(register_mean, on = 'registered_via', how = 'left').merge(payment_mean, on = 'payment_method_id', how = 'left')
dataset_train_v3 = dataset_train_v3.drop(columns = ['city', 'registered_via', 'payment_method_id'])

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score

X = dataset_train_v3.drop(columns = ['is_churn'])
Y = dataset_train_v3['is_churn']




from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import sklearn
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
nm_models_pre = []
nm_models_pre.append(('KNN', KNeighborsClassifier()))
nm_models_pre.append(('LR', LogisticRegression()))
nm_models_pre.append(('LDA', LinearDiscriminantAnalysis()))
nm_models_pre.append(('CART', DecisionTreeClassifier()))

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
results = []
names = []
for name, model in nm_models_pre:
 nm_cv_results = cross_val_score(model, X_train, Y_train, cv=10, scoring='neg_log_loss', n_jobs = -1)
 model.fit(X_train,Y_train);
 y_pred = model.predict(X_test)
 cm = confusion_matrix(Y_test, y_pred)
 acc = accuracy_score(Y_test, y_pred)
 print(cm)
 print(acc)
 results.append(nm_cv_results)
 names.append(name)
 print("Auc-ROC:")
 print(roc_auc_score(Y_test, y_pred))
 msg = "%s: %f (%f)" % (name, nm_cv_results.mean(), nm_cv_results.std())
 print(msg)
	
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import numpy
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
classifier = Sequential()
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'Adam', loss ='binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, Y_train, batch_size = 10, epochs = 5)
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)
acc = accuracy_score(Y_test, y_pred)
print(acc)
svc = svm.SVC(C=1, kernel='linear')
print("Auc-ROC:")
print(roc_auc_score(Y_test, y_pred))






# Feature Scaling for modelling purpose by using both min-max-scaling
from sklearn.preprocessing import MinMaxScaler
nm_X = pd.DataFrame(MinMaxScaler().fit_transform(X))
nm_X.columns = X.columns.values
nm_X.index = X.index.values

# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2, f_classif
nm_col = ['is_auto_renew', 'is_cancel']
nm_X_v2 = nm_X.drop(columns = nm_col)
nm_X_v3 = pd.DataFrame(nm_X, columns = nm_col)
nm_X_v4 = pd.DataFrame(SelectKBest(score_func=chi2, k='all').fit(nm_X_v3, Y).pvalues_ <= 0.05, columns = ['importance'])
nm_X_v4.index = nm_X_v3.columns.values
nm_X_v5 = pd.DataFrame(SelectKBest(score_func=f_classif, k='all').fit(nm_X_v2, Y).pvalues_ <= 0.05, columns = ['importance'])
nm_X_v5.index = nm_X_v2.columns.values
nm_X_v6 = pd.concat([nm_X_v4, nm_X_v5])
nm_selected = list(pd.Series(nm_X_v6[nm_X_v6['importance'] == 1].index.values))
nm_X_v7 = pd.DataFrame(nm_X, columns = nm_selected)

# Dimension Reduction
from sklearn.decomposition import PCA
pca = PCA()
pca.fit_transform(nm_X_v7)
np.cumsum(pca.explained_variance_ratio_)
nm_X_v8 = PCA(n_components=10).fit_transform(nm_X_v7)

# Split into train and test Set
from sklearn.model_selection import train_test_split
nm_X_train, nm_X_test, nm_Y_train, nm_Y_test = train_test_split(nm_X_v8, Y, test_size = 0.3, random_state = 0)

# Fit training set into different algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb

nm_models = []

nm_models.append(('LR', LogisticRegression()))
nm_models.append(('LDA', LinearDiscriminantAnalysis()))

ensembles = []

ensembles.append(('XGB', XGBClassifier()))

from sklearn.model_selection import cross_val_score
results = []
names = []
for name, model in nm_models:
  nm_cv_results = cross_val_score(model, nm_X_train, nm_Y_train, cv=10, scoring='neg_log_loss', n_jobs = -1)
  model.fit(nm_X_train,nm_Y_train)
  results.append(nm_cv_results)
  nm_y_pred = model.predict(nm_X_test)
  cm = confusion_matrix(nm_Y_test, nm_y_pred)
  acc = accuracy_score(nm_Y_test, nm_y_pred)
  print(acc)
  print(cm)
  names.append(name)
  print("AUC-ROC:")
  print(roc_auc_score(nm_Y_test, nm_y_pred))
  msg = "%s: %f (%f)" % (name, nm_cv_results.mean(), nm_cv_results.std())
  print(msg)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
nm_X_train = sc.fit_transform(nm_X_train)
nm_X_test = sc.transform(nm_X_test)
classifier = Sequential()
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'Adam', loss ='binary_crossentropy', metrics = ['accuracy'])
classifier.fit(nm_X_train, nm_Y_train, batch_size = 10, epochs = 5)
y_pred = classifier.predict(nm_X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(nm_Y_test, y_pred)
print(cm)
acc = accuracy_score(nm_Y_test, y_pred)
print(acc)
svc = svm.SVC(C=1, kernel='linear')
print("AUC-ROC:")
print(roc_auc_score(nm_Y_test, nm_y_pred))
	
	
    
results2 = []
names2 = []
for name2, model2 in ensembles:
  en_cv_results = cross_val_score(model2, nm_X_train, nm_Y_train, cv=10, scoring='neg_log_loss', n_jobs = -1)
  results2.append(en_cv_results)
  names2.append(name2)
  nm_y_pred = model.predict(nm_X_test)
  cm = confusion_matrix(nm_Y_test, nm_y_pred)
  acc = accuracy_score(nm_Y_test, nm_y_pred)
  print(acc)
  print(cm)
  print("AUC-ROC:")
  print(roc_auc_score(nm_Y_test, nm_y_pred))
  msg2 = "%s: %f (%f)" % (name2, en_cv_results.mean(), en_cv_results.std())
  print(msg2)

# Apply Grid Search on XGBoost since it returns the best result on Cross Validation among all models
from sklearn.model_selection import GridSearchCV
parameters = {"learning_rate": [0.1],
              "min_child_weight": [1],
              "max_depth": [9],
              "gamma": [0.1],
              "subsample": [0.8],
              "colsample_bytree": [0.8],
              "objective": ['binary:logistic']}
grid_search_XGB = GridSearchCV(estimator = XGBClassifier(), param_grid = parameters, scoring = "neg_log_loss", cv = 10, n_jobs = -1)
grid_result_XGB = grid_search_XGB.fit(nm_X_train, nm_Y_train)
print("Best: %f using %s" % (grid_result_XGB.best_score_, grid_result_XGB.best_params_)) 

# Evaluate tuned XGBoost model result on test dataset because it provides the best result
from sklearn.metrics import log_loss
nm_Y_predict = grid_result_XGB.predict_proba(nm_X_test)
logloss = log_loss(nm_Y_test, nm_Y_predict)  
print("Log Loss Score: %s" % (logloss))

[[263303   1789]
 [ 16554   9642]]
0.9370279585839444
Auc-ROC:
0.6806614304029657
KNN: -1.248614 (0.008413)
[[263911   1181]
 [ 20372   5824]]
0.9260079371618467
Auc-ROC:
0.6089344809109704
LR: -0.243184 (0.001616)


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[[260598   4494]
 [ 13810  12386]]
0.9371618466946802
Auc-ROC:
0.7279338363858557
LDA: -0.268407 (0.004003)
[[260963   4129]
 [  4151  22045]]
0.9715745241822527
Auc-ROC:
0.9129824842907631
CART: -0.974079 (0.018561)
Epoch 1/5
67968/67968 [==============================] - 109s 1ms/step - loss: 0.1280 - accuracy: 0.9521
Epoch 2/5
67968/67968 [==============================] - 92s 1ms/step - loss: 0.0896 - accuracy: 0.9688
Epoch 3/5
67968/67968 [==============================] - 91s 1ms/step - loss: 0.0884 - accuracy: 0.9693
Epoch 4/5
67968/67968 [==============================] - 95s 1ms/step - loss: 0.0875 - accuracy: 0.9697
Epoch 5/5
67968/67968 [==============================] - 90s 1ms/step - loss: 0.0868 - accuracy: 0.9699
[[260902   4190]
 [  4543  21653]]
0.9700193622806295
Auc-ROC:
0.9053853715650375
0.9351432259482024
[[261180   3912]
 [ 14980  11216]]
AUC-ROC:
0.7066999148063683
LR: -0.178730 (0.001039)
0.9359465546126171
[[260637   4455]
 [ 14203  11993]]
AUC-ROC:
0.72050625